# DSCI 525: Milestone 1

## Installing packages that are not included in the environment file

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} altair

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\yuzhe\miniconda3\envs\525

  added / updated specs:
    - altair


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    attrs-20.3.0               |     pyhd3deb0d_0          41 KB  conda-forge
    entrypoints-0.3            |  pyhd8ed1ab_1003           8 KB  conda-forge
    importlib-metadata-3.10.0  |   py39hcbf5309_0          27 KB  conda-forge
    jsonschema-3.2.0           |     pyhd8ed1ab_3          45 KB  conda-forge
    pyrsistent-0.17.3          |   py39hb82d6ee_2          92 KB  conda-forge
    zipp-3.4.1                 |     pyhd8ed1ab_0          11 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         839 KB

The following NEW 



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.0

Please update conda by running

    $ conda update -n base conda




altair-4.1.0         | 614 KB    | ########## | 100% 
altair-4.1.0         | 614 KB    | ########## | 100% 

pyrsistent-0.17.3    | 92 KB     |            |   0% 
pyrsistent-0.17.3    | 92 KB     | ########## | 100% 
pyrsistent-0.17.3    | 92 KB     | ########## | 100% 

zipp-3.4.1           | 11 KB     |            |   0% 
zipp-3.4.1           | 11 KB     | ########## | 100% 

jsonschema-3.2.0     | 45 KB     |            |   0% 
jsonschema-3.2.0     | 45 KB     | ########## | 100% 
jsonschema-3.2.0     | 45 KB     | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


## Library Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage
import dask.dataframe as dd
import gc
import sys
import dask
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
import altair as alt
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import dask.dataframe as dd
import pyarrow.feather as feather

In [3]:
%load_ext rpy2.ipython
%load_ext memory_profiler

C:\Users\yuzhe\miniconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


## 1. Team-work contract

- Please find the link to the teamwork contract [here](https://docs.google.com/document/d/1Sfs2RBib0FiGaDob7pGoitGPthRL-F32-dZo7A-aOhg/edit#heading=h.tgeu41bizsao).

## 2. Creating repository and project structure

- Please find the GitHub repository for the project [here](https://github.com/UBC-MDS/525_group18).

## 3. Downloading the Data

In [3]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare_rainfall/"

Get the necessary files from the figshareAPI for the rainfall article.

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]             
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

We are interested to download the `data.zip` file from the rainfall article from figshare.

We are also timing how long it takes to download the data through the figshareAPI

In [5]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 7.81 s, sys: 7.05 s, total: 14.9 s
Wall time: 1min 33s


Extracting the file from the `data.zip` folder

In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 23 s, sys: 3.99 s, total: 27 s
Wall time: 1min 1s


## 4. Comibining the data CSVs with Pandas and Dask

### 4.1 Using Pandas

In [10]:
%%time
%memit

com_dat_path = "figshare_rainfall/combined_data.csv"

files = glob.glob('figshare_rainfall/*NSW.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[^\/]+(?=\_d)', file)[0])
                for file in files)
              )
df.to_csv(com_dat_path)

peak memory: 156.71 MiB, increment: 0.04 MiB
Wall time: 7min 38s


In [8]:
%%sh
du -sh figshare_rainfall/combined_data.csv

5.6G	figshare_rainfall/combined_data.csv


In [9]:
%%time
df = pd.read_csv("figshare_rainfall/combined_data.csv")

CPU times: user 1min 15s, sys: 19.7 s, total: 1min 35s
Wall time: 4min 41s


In [10]:
print(f"The data Usage for combined dataframe in pandas is {df.memory_usage(index=True).sum()}")

The data Usage for combined dataframe in pandas is 3498199336


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float64
 2   lat_max        float64
 3   lon_min        float64
 4   lon_max        float64
 5   rain (mm/day)  float64
 6   model          object 
dtypes: float64(5), object(2)
memory usage: 3.3+ GB


In [12]:
sys.getsizeof(df)

11448609766

In [13]:
print(df.shape)

(62467843, 7)


In [14]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.811518,-34.86911,140.625,141.875,3.045650e-13,SAM0-UNICON
1,1889-01-02 12:00:00,-35.811518,-34.86911,140.625,141.875,3.572392e-04,SAM0-UNICON
2,1889-01-03 12:00:00,-35.811518,-34.86911,140.625,141.875,9.431562e-03,SAM0-UNICON
3,1889-01-04 12:00:00,-35.811518,-34.86911,140.625,141.875,9.663865e-02,SAM0-UNICON
4,1889-01-05 12:00:00,-35.811518,-34.86911,140.625,141.875,0.000000e+00,SAM0-UNICON


***Observations***

- Zhenrui：

    - Combining files:
        - peak memory: 2770.49 MiB, increment: 0.02 MiB
        - Wall time: 6min 56s
    - Reading combined file:
        - Wall time: 1min 03s
    - Combined Csv size
        - 5.7G - figshare_rainfall/combined_data.csv
    - System resource Usage:
        - Ram: 5.8 GB
        - Cache: 2.9 GB

- Bruhat:
    - Combining files:
        - peak memory: 154.38 MiB, increment: 0.00 MiB
        - Wall time: 7min 38s
    - Reading combined file:
        - Wall time: 4min 6s
    - Combined Csv size
        - 5.6G - figshare_rainfall/combined_data.csv
    - System resource Usage:
        - Ram: 5.7 GB
        - Cache: 2.9 GB

- Deepak: Panda's way of combining the different csv files did not work for me, my computer crushed while processing the code. Since I only have 4GB RAM available for memory.
    

### Using Dask

In [7]:

%%time
%memit
file_list = glob.glob('figshare_rainfall/*NSW.csv')

def read_and_label_csv(filename):
    '''
    This function loads the csv file and adds a 'Model' column
    based on the filename
    
    Parameters
    ----------
    filename: str
        relative path to the file 
    
    Returns
    -------
        df_csv, a dataframe
    '''

    # reads each csv file to a pandas.DataFrame
    df_csv = pd.read_csv(filename, index_col=0)
    df_csv['Model'] = re.findall(r'[^\/]+(?=\_d)',filename)[0]
    return df_csv 

# create a list of functions ready to return a DataFrame
dfs = [dask.delayed(read_and_label_csv)(fname) for fname in file_list]

# using delayed, assemble the pandas.DataFrames into a dask.DataFrame
ddf = dd.from_delayed(dfs)

ddf.to_csv("figshare_rainfall/combined_data_dask.csv", single_file=True)

peak memory: 346.29 MiB, increment: 0.00 MiB


error: unbalanced parenthesis at position 21

In [19]:
%%sh
du -sh figshare_rainfall/combined_data.csv

6.7G	figshare_rainfall/combined_data.csv


In [5]:
%%time
df = dd.read_csv("figshare_rainfall/combined_data.csv")

Wall time: 24.5 ms


In [6]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,figshare_rainfall\ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,figshare_rainfall\ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,figshare_rainfall\ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,figshare_rainfall\ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,figshare_rainfall\ACCESS-CM2


In [22]:
df.dtypes

time              object
lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

***Observations***

- Deepak：
    - Combining files:
        - peak memory: 175.45 MiB, increment: 0.37 MiB
        - Wall time: 7min 44s
    - Reading combined file:
        - Wall time: 444 ms
    - Combined Csv size
        - 6.0G - figshare_rainfall/combined_data_dask.csv
    - System resource Usage:
        - Ram: 4.5 GB
        - Cache: 1.5 GB

- Zhenrui：
    - Combining files:
        - peak memory: 174.59 MiB, increment: 0.00 MiB
        - Wall time: 7min 50s
    - Reading combined file:
        - Wall time: 22.7 ms
    - Combined Csv size
        - 6.7G - figshare_rainfall/combined_data_dask.csv
    - System resource Usage:
        - Ram: 4.5 GB
        - Cache: 1.5 GB   

By comparision, dask obviosly has the faster read speed.

## 5. Load the combined CSV to memory and perform a simple EDA

In [15]:
#Clearing out previous memory 
del df
gc.collect()

17

### 5.1 Dask Approach to reduce memory usage

In [16]:
%%time
%memit
df_dask = dd.read_csv(com_dat_path)

peak memory: 5342.96 MiB, increment: 0.33 MiB
CPU times: user 65.8 ms, sys: 480 ms, total: 546 ms
Wall time: 853 ms


In [35]:
df_dask.memory_usage(deep=True)

Dask Series Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: series-groupby-sum-agg, 391 tasks

In [18]:
print(df_dask.model.value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64


### 5.2 Loading just columns what we want approach to reduce memory usage.

In [31]:
#loading time and rain col to perform eda
df_reqcol = pd.read_csv(com_dat_path, usecols = ['time', 'rain (mm/day)'])
df_reqcol['time'] = pd.to_datetime(df_reqcol['time'])

In [32]:
df_reqcol.resample('M', on= 'time').mean()
#alt.Chart(df_dask).mark_point().encode()
#df.info()

,rain (mm/day)
time,
1889-01-31,2.380630
1889-02-28,2.553540
1889-03-31,1.704693
1889-04-30,1.577561
1889-05-31,1.957795
...,...
2014-08-31,1.601111
2014-09-30,1.375966
2014-10-31,1.744105


## 6. Perform a simple EDA in R

### 6.1 Using Pandas

In [14]:
%%time
%%memit
#simple pandas - This is how we do normally ,which means we are loading the entire data to the memory
# just loading 10 million rows
df = pd.read_csv("figshare_rainfall/combined_data.csv")

peak memory: 8706.05 MiB, increment: 8367.50 MiB
Wall time: 1min 15s


In [ ]:
#%%time
#%%R -i df

#start_time <- Sys.time()
#library(dplyr)
#result <- df %>% count(model)
#end_time <- Sys.time()
#print(end_time - start_time)

***does not success in our computers***

### 6.2 Arrow + parquet

In [7]:
%%time
%%memit
# Read data file and prepare arrow table
dataset = ds.dataset("figshare_rainfall/combined_data.csv", format="csv")
table = dataset.to_table()

peak memory: 5147.54 MiB, increment: 4971.06 MiB
Wall time: 29 s


In [25]:
%%time
pq.write_to_dataset(table, 'figshare_rainfall/combined_data.parquet')

Wall time: 12.3 s


In [26]:
%%sh
du -sh figshare_rainfall/combined_data.csv
du -sh figshare_rainfall/combined_data.parquet

6.7G	figshare_rainfall/combined_data.csv
542M	figshare_rainfall/combined_data.parquet


In [3]:
%%time
%%R

library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_parquet("figshare_rainfall/combined_data.parquet/e898e12c7a804d5bad47296499329ccc.parquet")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

R[write to console]: 
Attaching package: 'arrow'


R[write to console]: The following object is masked from 'package:utils':

    timestamp




[1] "tbl_df"     "tbl"        "data.frame"


R[write to console]: 
Attaching package: 'dplyr'


R[write to console]: The following objects are masked from 'package:stats':

    filter, lag


R[write to console]: The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




# A tibble: 27 x 2
   model                                       n
   <chr>                                   <int>
 1 "figshare_rainfall\\ACCESS-CM2"       1932840
 2 "figshare_rainfall\\ACCESS-ESM1-5"    1610700
 3 "figshare_rainfall\\AWI-ESM-1-1-LR"    966420
 4 "figshare_rainfall\\BCC-CSM2-MR"      3035340
 5 "figshare_rainfall\\BCC-ESM1"          551880
 6 "figshare_rainfall\\CanESM5"           551880
 7 "figshare_rainfall\\CMCC-CM2-HR4"     3541230
 8 "figshare_rainfall\\CMCC-CM2-SR5"     3541230
 9 "figshare_rainfall\\CMCC-ESM2"        3541230
10 "figshare_rainfall\\EC-Earth3-Veg-LR" 3037320
# ... with 17 more rows
Time difference of 22.53359 secs
Wall time: 23.1 s


### 6.3 Arrow + feather

In [8]:
%%time
feather.write_feather(table, 'figshare_rainfall/combined_data.feather')

Wall time: 2.52 s


In [9]:
%%sh
du -sh figshare_rainfall/combined_data.feather

1.1G	figshare_rainfall/combined_data.feather


In [ ]:
%%time
%%R

library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("figshare_rainfall/combined_data.feather")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

Comparing the storage usage and time spending, arrow and parquet is the best option.